In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import collections
import copy
import numbers
import os
import sys
import uuid
from itertools import product, combinations
from importlib import reload
sys.path.append(r'C:\dev\projects\weedget')

import ipywidgets as w
import ipyvuetify as v
import ipyvue as vue
import numba
import numpy as np
import pandas as pd
import traitlets

from scipy.special import comb
from plotly import graph_objects as go, express as px
from plotly.subplots import make_subplots
from weedget import base as b, components as c
reload(c)
rng = np.random.default_rng(1988)
pd.options.plotting.backend = "matplotlib"

In [32]:
n_cols = 3
n_rows = 10
start = '1900-01-01'

df = pd.DataFrame(
    data=rng.normal(size=(n_rows, n_cols)),
    index=pd.DatetimeIndex(pd.date_range(start=start, periods=n_rows)),
    columns=[uuid.uuid4().hex[:10] for i in range(n_cols)]
)

In [4]:
features = pd.read_csv('../data/Raw_data.csv', index_col=0, parse_dates=True).drop('US_CBO_Nairu', axis=1).dropna(how='all', axis=1).dropna(how='all')

In [ ]:

def pairwise_prod2(df):
    n_features = df.shape[1]
    arr = df.to_numpy().T.astype(np.float32)
    indices = np.concatenate([np.arange(n_features * (i - 1) + i, n_features * i) for i in range(1, n_features)])
    products = (arr[:, np.newaxis] * arr).reshape(n_features ** 2, -1)[indices]
    col_names = pd.DataFrame([*product(df.columns, df.columns)]).iloc[indices]
    return pd.DataFrame(products.T, index=df.index, columns=pd.Index(col_names.iloc[:, 0] + '*' + col_names.iloc[:, 1]))
    
    
    
    

In [ ]:
def get_products(arr, cols):
    n = len(arr)
    prods = producs = np.concatenate([arr[i] * arr[i+1:] for i in range(n - 1)]).T
    cols = np.array([*combinations(cols, 2)])
    cols = np.char.add(np.char.add(cols[:, 0], '*'), cols[:, 1])
    return prods, cols

import math

@numba.njit
def get_products2(arr, cols):
    n = len(arr)
    n_prods = int(math.factorial(n) / (2 * math.factorial(n - 2)))
    prods_arr = np.empty(shape=(n_prods, arr.shape[1]))
    cols_arr = np.chararray(shape=n_prods, itemsize=100)
    start = 0
    for i in range(n - 1):
        n_comb = n - i - 1
        prods_arr[start:start + n_comb] = arr[i] * arr[i+1:]
        cols_arr[start:start + n_comb] = np.char.add(np.char.add(cols[i], b'*'), cols[i+1:])
        start += n_comb
    
    return prods_arr, cols_arr


def get_products(arr, cols):
    n = len(arr)
    prods = producs = np.concatenate([arr[i] * arr[i+1:] for i in range(n - 1)]).T
    cols = np.array([*combinations(cols, 2)])
    cols = np.char.add(np.char.add(cols[:, 0], '*'), cols[:, 1])
    return prods, cols


def pairwise_prod(X: pd.DataFrame, **kwargs):
    n_features = X.shape[1]
    arr = X.to_numpy().T.astype(np.float32)
    prods, cols = get_products(arr, df.columns)
    return pd.DataFrame(prods, index=X.index, columns=cols)    

In [ ]:
pairwise_prod2(df)

In [ ]:
from tsmoothie.smoother import (
    ExponentialSmoother, 
    ConvolutionSmoother,
    LowessSmoother,
    KalmanSmoother,
    SplineSmoother,
    SpectralSmoother,
    GaussianSmoother,
    BinnerSmoother,
    PolynomialSmoother
)

In [ ]:
s = features['US_Bond_10y'].dropna()

In [ ]:
sm = ExponentialSmoother(window_len=20, alpha=0.1)

In [ ]:
sm = ConvolutionSmoother(window_len=20, window_type='ones')

In [ ]:
smoothers = {
    'conv': ConvolutionSmoother(window_len=20, window_type='ones'),
    'exp': ExponentialSmoother(window_len=20, alpha=0.1),
    'lowess': LowessSmoother(smooth_fraction=0.2), 
    'spline': SplineSmoother(spline_type='natural_cubic_spline', n_knots=50),
    'spectral': SpectralSmoother(smooth_fraction=0.7, pad_len=5),
    'gaussian': GaussianSmoother(sigma=0.5, n_knots=50),
    'binner': BinnerSmoother(n_knots=50),
    'poly': PolynomialSmoother(5)    
}

In [ ]:
smoothed = []
for key, value in smoothers.items():
    smoothed.append(pd.Series(value.smooth(s).smooth_data.squeeze(), index=s.index[-value.smooth_data.shape[1]:]))
smoothed = pd.concat([*smoothed], axis=1).set_axis(smoothers, axis=1)
plot_data = pd.concat([s, smoothed], axis=1)

In [ ]:
fig = px.line(plot_data, x=plot_data.index, y=plot_data.columns)

In [ ]:
fig

In [ ]:
sm.smooth(s).smooth_data.shape

In [ ]:
features['US_Bond_10y']

In [ ]:
s.rolling(20).mean().dropna().plot()
pd.Series(sm.smooth(s).smooth_data.squeeze(), index=s.index[-sm.smooth_data.shape[1]:]).plot()

In [ ]:
pd.Series(sm.smooth(s).smooth_data.squeeze(), index=s.index[-sm.smooth_data.shape[1]:])

In [ ]:
pd.DataFrame(sm.smooth(s).smooth_data.T, index=df.index[-es.smooth_data.shape[1]:], columns=df.columns)

In [ ]:
df.shape[0]

In [ ]:
es.window_len >= df.shape[0]

In [ ]:
pd.Index(['a', 'b', 'c']) + pd.Index(['a', 'b', 'c'])

In [ ]:
df.ewm(alpha=0.1).mean()[100:]

In [5]:
from weedget import transformers as t

In [8]:
from weedget.transformers.pd impo

AttributeError: module 'weedget.transformers' has no attribute 'base'

In [ ]:
t.EWMZScore().set_columns(suffix='FUUUU', sep='___').fit_transform(features['JPN_Nikkei_PE'].dropna())

In [ ]:
(features['JPN_Nikkei_PE'].dropna() - features['JPN_Nikkei_PE'].dropna().ewm(alpha=0.1).mean()) / features['JPN_Nikkei_PE'].dropna().ewm(alpha=0.1).std()

In [ ]:
from weedget.transformers.preprocessing import StandardScaler

In [ ]:
StandardScaler().fit_transform(df)

In [ ]:
from functools import wraps
from sklearn.base import BaseEstimator, TransformerMixin


def check_in(X, **kwargs):
    if isinstance(X, pd.Series):
        return X.to_frame()
    return X


def transformer(klass):
    '''
    Decorator used to make sklearn transformers work with pandas data frame 
    '''
    
    class Wrapper(klass, Transformer):
        def fit(self, X, y=None, **fit_params):
            X  = check_in(X)
            super().fit(X, y=None, **fit_params)
            self._columns = self._get_columns(X.columns)
            return self
            
        def transform(self, X):
            X = check_in(X)
            X_tr = super().transform(X)
            pd.DataFrame(X_tr, index=X.index[-len(X_tr):], columns=self._columns)
            return pd.DataFrame(X_tr, index=X.index[-len(X_tr):], columns=self._columns)
            
        def fit_transform(self, X, y=None, **fit_params):
            X  = check_in(X)
            X_tr = super().fit_transform(X, y=None, **fit_params)
            self._columns = self._get_columns(X.columns)            
            return pd.DataFrame(X_tr, index=X.index[-len(X_tr):], columns=self._columns)
            

    Wrapper.__name__ = klass.__name__
    Wrapper.__qualname__ = klass.__qualname__
    Wrapper.__module__ = klass.__module__
    return Wrapper


class Transformer(TransformerMixin, BaseEstimator):
    _sep = '->'
    
    def fit(self, X: pd.DataFrame, y=None, **fit_params):
        X  = check_in(X)
        self._fit(X, y, **fit_params)
        self._columns = self._get_columns(X.columns)
        return self
    
    def transform(self, X):
        X = check_in(X)
        X_tr = self._transform(X)
        return aa
    
    def _get_columns(self, columns: pd.Index):
        prefix = self._prefix(columns)
        suffix = self._suffix(columns)
        new_columns = prefix + self._sep + columns if prefix else columns
        new_columns = new_columns + self._sep + suffix if suffix else new_columns
        return new_columns
        
    def _prefix(self, columns: pd.Index):
        pass
    
    def _suffix(self, columns: pd.Index):
        return self._cls_name
    
    @property
    def _cls_name(self):
        return self.__class__.__name__


class PandasMethod:
    def _fit(self, X, y=None, **fit_params):
        self.attr_ = getattr(X, self._attr)(**{self._param: getattr(self, self._param)})
        return self
    
    def _transform(self, X, y=None):
        return getattr(self.attr_, self._fn)()
    
    def _suffix(self, columns):
        return f'{self._cls_name}({self._param}={getattr(self, self._param)})'
    
    
class Rolling(PandasMethod):
    _attr = 'rolling'
    _param = 'window'
    def __init__(self, window=10):
        self.window = window
        

class EWM(PandasMethod):
    _attr = 'ewm'
    _param = 'alpha'
    def __init__(self, alpha=0.1):
        self.alpha = alpha
        
        
class Expanding(PandasMethod):
    _attr = 'expanding'
    _param = 'min_periods'
    def __init__(self, min_periods=50):
        self.min_periods = min_periods
    

class ZScore:
    def _transform(self, X, y=None):
        return (X - self.attr_.mean()) / self.attr_.std()
    

class RollingMean(Rolling, Transformer):
    _fn = 'mean'
    

class RollingStd(Rolling, Transformer):
    _fn = 'std'
    

class RollingZScore(ZScore, EWM, Transformer):
    pass


class EWMMean(Rolling, Transformer):
    _fn = 'mean'
    

class EWMStd(Rolling, Transformer):
    _fn = 'std'
    

class EWMZScore(ZScore, EWM, Transformer):
    pass


class ExpandingMean(Expanding, Transformer):
    _fn = 'mean'
    

class ExpandingStd(Expanding, Transformer):
    _fn = 'std'
    

class ExpandingZScore(ZScore, Expanding, Transformer):
    pass

        

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

SS = transformer(StandardScaler)
SI = transformer(SimpleImputer)

In [ ]:
SS = t.transformer(StandardScaler)

In [ ]:
SS.__module__


In [ ]:
s1= pd.Series([1,2,3, 4, np.nan, 6])

In [ ]:
s1

In [ ]:
r = s1.rolling(3)

In [ ]:
s1.fillna(r.mean().ffill())

In [ ]:
r.mean()

In [ ]:
features.mean()

In [ ]:
features.fillna()

In [ ]:
(features['JPN_Nikkei_PE'].dropna() - features['JPN_Nikkei_PE'].dropna().mean()) / features['JPN_Nikkei_PE'].dropna().std()

In [ ]:
SS().fit_transform(features['JPN_Nikkei_PE'].dropna())

In [ ]:
features.expanding()

In [ ]:
features.dropna()

In [ ]:
(features['JPN_Nikkei_PE'].dropna() - features['JPN_Nikkei_PE'].dropna().ewm(alpha=0.1).mean()) / features['JPN_Nikkei_PE'].dropna().ewm(alpha=0.1).std()

In [ ]:
EWMZScore().fit_transform(features['JPN_Nikkei_PE'].dropna())

In [ ]:
(df  - df.rolling(10).mean()) / df.rolling(10).std()

In [ ]:
s.rolling(10).mean()

In [ ]:
self = RollingMean(10)

In [ ]:
{self._param: getattr(self, self._param)}

In [ ]:
RollingMean(10).fit_transform(s)

In [ ]:
def rolling_mean(X, window, **kwargs):
    return X.rolling(window).mean()


def rolling_std(X, window, **kwargs):
    return X.rolling(window).std()


def ewm_mean(X, alpha, **kwargs):
    return X.ewm(alpha=alpha).mean()
    

    
def ewm_mean(X, alpha, **kwargs):
    return X.ewm(alpha=alpha).mean()

In [ ]:
fracdiff_weights(0.5)

In [46]:
def lagdf(df, max_lag, start=0):
    # this version of lagdf is a multiindex
    try:
        df = df.to_frame()
    except AttributeError:
        pass
    return pd.concat({i: df.shift(i) for i in range(start, max_lag + 1)}, axis=1). \
        reorder_levels([1, 0], axis=1). \
        rename_axis(default_names[1:], axis=1)

In [47]:
def fracdiff_weights(d, max_lag=5, threshold=0):
    '''
    :param d: degrees of fractional difference, can be a 
    '''

    if isinstance(d, numbers.Number):
        d = [d]
    elif isinstance(d, dict):
        d = [*d.values()]

    w_mat = np.empty(shape=(max_lag + 1, len(d)))

    for i in range(w_mat.shape[1]):
        w, k = [1], 1

        while k <= max_lag:
            if np.abs(w[-1]) <= threshold:
                break
            w.append(- w[-1] * (d[i] - k + 1) / k)
            k += 1

        # this is to add multiple zeros so that all the rows have the same length
        to_add = max_lag + 1 - len(w)
        if to_add > 0:
            for j in range(to_add):
                w.append(0)

        w_mat[:, i] = w

    return w_mat.squeeze()


def fracdiff(X, d, max_lag=5, threshold=0, dropna=True):
    '''
    :param x: a pandas series or DataFrame
    :param d: int, float, list or dict: degree of difference operator (1 - L) ** d
              if dict must have all the keys equal to the the columns of X
              if list must have the same length as X
    :param max_lag: maximum number of lags to use in the Taylor expansion
    :param threshold: threshold that defines when to break the recursion for the weights in the taylor expansion
    :return:
    '''

    if isinstance(X, pd.Series):
        X = X.to_frame()
    if isinstance(d, list):
        d = {col: value for col, value in zip(X, d)}
    elif isinstance(d, dict):
        d = pd.Series(d).reindex(X.columns).to_dict()
    # getting weights for taylor expansion
    w = pd.DataFrame(fracdiff_weights(d, max_lag=max_lag, threshold=threshold), columns=X.columns)

    # creating dataframe with lags
    # if dropna:
    df_lags = lagdf(X, max_lag, start=0).dropna().sort_index(axis=1)  # level=[0, 1])
    # else:
    #     df_lags = utils.lagdf(X, max_lag, start=0).sort_index(axis=1)

    # in case of one feature do not call groupby
     # getting weights for taylor expansion
    w = fracdiff_weights(d, max_lag=max_lag, threshold=threshold)
    if X.shape[1] == 1:
        return df_lags @ w
    
     # getting weights for taylor expansion
    w = pd.DataFrame(w, columns=X.columns)
    df_group = df_lags.groupby(level=0, axis=1)

    result = {}
    for index, group in df_group:
        # group.sort_index(axis=1, level=0)
        result.setdefault(index, group.dropna() @ w[index].to_numpy())

    return pd.DataFrame(result).squeeze()

In [110]:
def fracdiff_weights(d, max_lag=5, threshold=0):

    if isinstance(d, numbers.Number):
        d = [d]
    if isinstance(d, dict):
        d = list(d.values())

    w_mat = np.empty(shape=(max_lag + 1, len(d)))

    for i in range(w_mat.shape[1]):
        w, k = [1], 1

        while k <= max_lag:
            if np.abs(w[-1]) <= threshold:
                break
            w.append(- w[-1] * (d[i] - k + 1) / k)
            k += 1

        # this is to add multiple zeros so that all the rows have the same length
        to_add = max_lag + 1 - len(w)
        if to_add > 0:
            for j in range(to_add):
                w.append(0)

        w_mat[:, i] = w

    return w_mat.squeeze()


def fracdiff(X, d, max_lag=5, threshold=0, dropna=True):
    '''
    :param x: a pandas series or DataFrame
    :param d: int, float, list or dict: degree of difference operator (1 - L) ** d
              if dict must have all the keys equal to the the columns of X
              if list must have the same length as X
    :param max_lag: maximum number of lags to use in the Taylor expansion
    :param threshold: threshold that defines when to break the recursion for the weights in the taylor expansion
    :return:
    '''

    if isinstance(X, pd.Series):
        X = X.to_frame()
    if isinstance(d, numbers.Number):
        d = [d] * X.shape[1]
    if isinstance(d, list):
        d = {col: value for col, value in zip(X, d)}

    # getting weights for taylor expansion
    w = pd.DataFrame(fracdiff_weights(d, max_lag=max_lag, threshold=threshold), columns=X.columns)

    # creating dataframe with lags
    # if dropna:
    df_lags = lagdf(X, max_lag, start=0).dropna().sort_index(axis=1)  # level=[0, 1])
    # else:
    #     df_lags = utils.lagdf(X, max_lag, start=0).sort_index(axis=1)

    # in case of one feature do not call groupby
    if df_lags.shape[1] == max_lag + 1:
        return df_lags @ w.to_numpy()

    df_group = df_lags.groupby(level=0, axis=1)

    result = {}
    for index, group in df_group:
        # group.sort_index(axis=1, level=0)
        result.setdefault(index, group.dropna() @ w[index].to_numpy())

    return pd.DataFrame(result).squeeze()

In [111]:
fracdiff(df, 0.5, dropna=False)

9e2f897c09  9f23249be4  c95d3c5331
1900-01-06   -0.072587   -2.075767   -1.002290
1900-01-07   -0.875388    0.398910    1.040041
1900-01-08   -0.517907    0.555064    1.005049
1900-01-09   -0.355278   -1.379952   -0.107236
1900-01-10    0.518595    0.295320    0.525392

In [ ]:
np.full(shape=)

In [83]:
(x3d * w).sum(axis=2)[-max_lag:]

array([[-2.07576685, -1.00229041, -0.07258702],
       [ 0.39891021,  1.04004095, -0.87538824],
       [ 0.55506446,  1.00504901, -0.51790713],
       [-1.37995168, -0.10723565, -0.35527805],
       [ 0.29531998,  0.52539164,  0.51859459]])

In [87]:
X = df
d = 0.5
d = {'c95d3c5331': 0.5, '9e2f897c09': 0.7, '9f23249be4': 0.6}
max_lag = 5 
threshold = 0

In [88]:
pd.Series(d).reindex(X.columns).to_dict()

{'9f23249be4': 0.6, 'c95d3c5331': 0.5, '9e2f897c09': 0.7}

{'c95d3c5331': 0.5, '9e2f897c09': 0.7, '9f23249be4': 0.6}

In [119]:
def fracdiff_weights(d, max_lag=5, threshold=0):
    '''
    :param d: degrees of fractional difference, can be a 
    '''

    if isinstance(d, numbers.Number):
        d = [d]

    w_mat = np.empty(shape=(len(d), max_lag + 1))

    for i in range(w_mat.shape[0]):
        w, k = [1], 1

        while k <= max_lag:
            if np.abs(w[-1]) <= threshold:
                break
            w.append(- w[-1] * (d[i] - k + 1) / k)
            k += 1

        # this is to add multiple zeros so that all the rows have the same length
        to_add = max_lag + 1 - len(w)
        if to_add > 0:
            for j in range(to_add):
                w.append(0)

        w_mat[i, :] = w

    return w_mat.squeeze()


def fracdiff(X, d, max_lag=5, threshold=0, dropna=False):

    '''
    :param x: a pandas series or DataFrame
    :param d: int, float, list or dict: degree of difference operator (1 - L) ** d
              if dict must have all the keys equal to X.columns
              if list must have the same length as X
    :param max_lag: maximum number of lags to use in the Taylor expansion
    :param threshold: threshold that defines when to break the recursion for the weights in the taylor expansion
    :return:
    '''

    if isinstance(X, pd.Series):
        X = X.to_frame()
    
    if isinstance(d, dict):
        d = pd.Series(d).reindex(X.columns).to_list()
    
    # getting weights for taylor expansion
    w = fracdiff_weights(d, max_lag=max_lag, threshold=threshold)
    
    arr = np.asarray(X)
    # x3d = np.full(shape=[*arr.shape, max_lag + 1], fill_value=np.nan)
    x3d = np.empty(shape=[*arr.shape, max_lag + 1])
    x3d[..., 0] = arr
    for i in range(1, max_lag + 1):
        # x3d[i:, :, i] = np.roll(arr, i, axis=0)[i:]
        x3d[..., i] = np.roll(arr, i, axis=0)
    fdiff = (x3d * w).sum(axis=2)[-max_lag:]
    
    res = pd.DataFrame(fdiff, index=X.index[-len(fdiff):], columns=X.columns)
    if not dropna:
        res = res.reindex(X.index)
    return res

In [125]:
from weedget.transformers.shift import FracDiff

ImportError: cannot import name 'FracDiff' from 'weedget.transformers.shift' (C:\dev\projects\weedget\weedget\transformers\shift.py)

In [23]:
d

0.5

In [20]:
w = fracdiff_weights(d, max_lag=max_lag, threshold=threshold)

In [22]:
w.shape

(6,)

In [122]:
@transformer
class FracDiffTransformer:

    def __init__(
        self, 
        d=2, 
        max_lag=5, 
        threshold=0
    ):
        self.d = d
        self.max_lag = max_lag
        self.threshold = threshold

        
    def fit(self, X, y=None, **fit_params):
        d = self.d 
        if isinstance(d, dict):
            d = pd.Series(d).reindex(X.columns).to_list()
        
        self.w_ = fracdiff_weights(d, max_lag=max_lag, threshold=threshold)
        return self
    
    def transform(X, y=None):
        arr = np.asarray(X)
        # x3d = np.full(shape=[*arr.shape, max_lag + 1], fill_value=np.nan)
        x3d = np.empty(shape=[*arr.shape, max_lag + 1])
        x3d[..., 0] = arr
        for i in range(1, max_lag + 1):
            # x3d[i:, :, i] = np.roll(arr, i, axis=0)[i:]
            x3d[..., i] = np.roll(arr, i, axis=0)
        fdiff = (x3d * w).sum(axis=2)[-max_lag:]

        # res = pd.DataFrame(fdiff, index=X.index[-len(fdiff):], columns=X.columns)
        res = pd.DataFrame(index=X.index[-len(fdiff):], columns=X.columns)
        res[-self.max_lag:] = fdiff
        return res
    


NameError: name 'transformer' is not defined

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame(np.random.normal(size=(100, 10)))
df.ewm(alpha=0.1).agg(lambda x: x)

In [ ]:
feature = features['US_Equity_Fut'].dropna()
param = 50

In [ ]:
feature.rolling(param).mean().plot()

In [ ]:
((feature - feature.rolling(param).mean()) / feature.rolling(param).std()).plot()

In [ ]:
((feature - feature.shift(1).rolling(param).std()) / feature.shift(1).rolling(param).std()).plot()

In [ ]:
feature.rolling(param).mean().diff().plot()

In [ ]:
feature.diff().rolling(param).mean().plot()

In [ ]:
((feature - feature.rolling(param).mean()) / feature.rolling(param).std()).diff().plot()

In [ ]:
((feature.diff() - feature.diff().rolling(param).mean()) / feature.diff().rolling(param).std()).plot()

In [ ]:
df.ewm(alpha=0.1).agg(lambda x: x)

In [ ]:
# built in
from typing import Optional

# third party
import numpy as np
import pandas as pd

from scipy import stats
from sklearn.base import TransformerMixin, BaseEstimator 


# Note:
# the reason why the prefix, suffix and sep have no effect for is because I override the _get_columns method
# which doesn't use them at all

class NaFilter(TransformerMixin, BaseEstimator):
    '''
    Filters the columns if the number of nan is greater than  n
    '''
    def __init__(self, n=100):
        self.n = n

    def fit(self, X, y=None, **kwargs):
        n  = self.n * len(X) if isinstance(self.n, float) else self.n
        self.mask_ = X.isna().sum(axis=0) < n
        return self
    
    def transform(self, X, y=None, **kwargs):
        return X.loc[:, self.mask_]

In [ ]:
class EWM(TransformerMixin, BaseEstimator):
    

In [ ]:
def fn(x):
    print(x)
    return 3
    

ewm = features.ewm(alpha=0.8)

In [ ]:
ewm.agg(fn)

In [ ]:
NaFilter(n=0.5).fit_transform(features)

In [ ]:
nan_filter = NaFilter()

In [ ]:

nan_filter.mask_

In [ ]:
nan_filter.fit_transform(features)

In [ ]:
features.dropna(how='all')

In [ ]:
dfj = b.df_to_json(features, A())

In [ ]:
c.DataFrameCombo(df=features.iloc[:, :100], empty=True)

In [ ]:
d = {
    df.columns[0]: "{:.2%}" 
}

In [ ]:
class DataFrameJS(b.BaseTemplate):
    """
    Vuetify DataTable rendering of a pandas DataFrame
    Note: using template_file does not work with autoreload because internally the template file is read and saved in a registry when first loaded.
          The only workaround was to restart the kernel
    Args:
        data (DataFrame) - the data to render
        title (str) - optional title
    """

    df = b.DataFrame(pd.DataFrame()).tag(sync=True)
    df_orig = b.DataFrame(pd.DataFrame()).tag(sync=True)
    formats = traitlets.Any().tag(sync=True)
    selected_cols = traitlets.List([]).tag(sync=True)
    search = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('DataFrame').tag(sync=True)
    clicked_row = traitlets.Dict({}).tag(sync=True)

    template = c.get_template('../templates/DataFrameJS.vue')

    def __init__(
            self,
            *args,
            **kwargs
    ):

        df = kwargs.pop('df').copy(deep=True)
        df.columns = df.columns.astype(str)
        self.df = df
        self.df_orig = df.copy()
        self.dtypes = df.reset_index().dtypes.to_dict()
        super().__init__(*args, **kwargs)

In [ ]:
js = DataFrameJS(df=df, formats='{:.2%}')
js

In [ ]:
d = DataFrame(df=iris)
d

In [ ]:
iris.index.nlevels

In [ ]:

class Filter(v.VuetifyTemplate):
    column = traitlets.Unicode('').tag(sync=True)
    columns = traitlets.List([]).tag(sync=True)
    conditions = traitlets.List(['==', '!=', '>', '<', '>=', '<=']).tag(sync=True)

    priority = traitlets.Int(1).tag(sync=True)
    condition = traitlets.Unicode('>').tag(sync=True)
    logic = traitlets.Unicode('AND').tag(sync=True)
    logics = traitlets.List(['AND', 'OR']).tag(sync=True)
    value = traitlets.Any().tag(sync=True)
    template = c.get_template('../templates/Filter.vue').tag(sync=True)
f = Filter()
f

In [ ]:
class FilterContainer(v.VuetifyTemplate):
    columns = traitlets.List([]).tag(sync=True)
    last_id = traitlets.Int(0).tag(sync=True)
    filters = traitlets.List([{'id': 0}]).tag(sync=True)
    template = c.get_template('../templates/FilterContainer.vue')
    components = traitlets.Dict({
        'mb-filter': c.get_template('../templates/Filter.vue', False)
    }).tag(sync=True, **v.VuetifyTemplate.class_component_serialization)

In [ ]:
columns = ['a', 'b', 'c']

filters = [
    {
        'id': 0,
        'col': '',
        'cond': '',
        'val': 'val',
        'logic': 'AND'
    }
]
fc = FilterContainer(columns=columns)
fc

In [ ]:
vue.VueComponentRegistry.vue_component_registry = {}
components = {
        'mb-filter-container': c.get_template('../templates/FilterContainer.vue', False),
        'mb-filter': c.get_template('../templates/Filter.vue', False)
    }

for name, string in components.items():
    vue.register_component_from_string(name, string)

In [ ]:
class DataFrameCombo(b.BaseTemplate):
    """
    Vuetify DataTable rendering of a pandas DataFrame
    Note: using template_file does not work with autoreload because internally the template file is read and saved in a registry when first loaded.
          The only workaround was to restart the kernel
    Args:
        data (DataFrame) - the data to render
        title (str) - optional title
    """

    df = b.DataFrame(pd.DataFrame()).tag(sync=True)
    df_orig = b.DataFrame(pd.DataFrame()).tag(sync=True)
    formats = traitlets.Any().tag(sync=True)
    selected_cols = traitlets.List([]).tag(sync=True)
    search = traitlets.Unicode('').tag(sync=True)
    title = traitlets.Unicode('DataFrame').tag(sync=True)
    clicked_row = traitlets.Dict({}).tag(sync=True)
    dialog = traitlets.Bool(False).tag(sync=True)
    table_key = traitlets.Int(0).tag(sync=True)

    template = c.get_template('../templates/DataFrameCombo.vue')

    def __init__(
            self,
            *args,
            **kwargs
    ):

        df = kwargs.pop('df').copy(deep=True)
        df.columns = df.columns.astype(str)
        self.df = df
        self.df_orig = df.copy()
        self.dtypes = df.reset_index().dtypes.to_dict()
        super().__init__(*args, **kwargs)

In [ ]:
n_cols = 5
n_rows = 10
start = '1900-01-01'

df = pd.DataFrame(
    data=rng.normal(size=(n_rows, n_cols)),
    index=pd.DatetimeIndex(pd.date_range(start=start, periods=n_rows)),
    columns=[uuid.uuid4().hex[:10] for i in range(n_cols)]
)

In [ ]:
dfc = DataFrameCombo(df=df, formats="{:.2%}")
dfc

In [ ]:
df[df[df.columns[0]] >0]

In [ ]:
df.reset_index().to_dict('records')

In [ ]:
class App(v.VuetifyTemplate):
    dialog = traitlets.Bool(False).tag(sync=True)
    columns = traitlets.List([]).tag(sync=True)
    template = c.get_template('../templates/DataFrameCombo.vue')

In [ ]:
a = App(columns=['a', 'b', 'c', 'd'])
a

In [ ]:
a.columns

In [ ]:
fc.filters

In [ ]:
def fn(e):
    print(e)

In [ ]:
fc.observe(fn, names='filters')

In [ ]:

v.Container(children=[
    'AAA',
    fs
])

In [ ]:
vue.VueComponentRegistry.vue_component_registry

In [ ]:
import ipyvue as vue
import traitlets

class Parent(vue.VueTemplate):
    template = traitlets.Unicode('''
        <template>
            <div>
                <div>parent</div>
                <full-vue />
            </div>
        </template>
    ''').tag(sync=True)
    
    myprop = traitlets.Unicode('hello').tag(sync=True)
    
    components = traitlets.Dict({'full-vue': '''
        <template>
            <div>
                Prop from parent: 
                <h3> AAAA </h3>
            </div>
        </template>
        <script>
            export default {
                props: ['someProp']
            }
        </script>
    '''}).tag(sync=True)

parent = Parent()
parent

In [ ]:
parent.myprop = 'hi'

In [ ]:
parent.components

In [ ]:
vue.register_component_from_string(
    'g-sub',
    '''
        <template>
            <span>sub-component</span>
        </template
''')

vue.register_component_from_string(
    'g-full-vue',
    '''
        <template>
            <div>
                Prop from parent (g): 
                <h3> {{ someProp}} </h3>
                <button @click="changeProp()">change</button>
                <div>
                    sub: <g-sub></g-sub>
                </div>
            </div>
        </template>
        <script>
            module.exports = {
                props: ['someProp'],
                methods: {
                    changeProp() {
                        this.$emit('update:someProp', 'new value')
                    }
                }
            }
        </script>
        <style scoped>
            h3 {
                color: blue
            }
        </style>
''')

class GlobalExample(vue.VueTemplate):
    template = traitlets.Unicode('''
        <template>
            <div>
                <div>parent</div>
                <g-full-vue :some-prop.sync="myprop"></g-full-vue>
            </div>
        </template>
    ''').tag(sync=True)
    
    myprop = traitlets.Unicode('hello').tag(sync=True)
    
globalExample = GlobalExample()
globalExample

In [ ]:
vue.Html(tag='h3', children=['test scoped'])

globalExample.myprop

globalExample.myprop = 'hi'

import ipyvuetify as v
from traitlets import Unicode, Dict, List
import ipywidgets as w

class Testing(v.VuetifyTemplate):
    template = Unicode("""
    <template>
        <test-comp :viewer="some_widget"></test_comp>
    </template>
    """).tag(sync=True)
    
    components = Dict({
        'test-comp': """
            <template>
              <v-card>
                <v-card-title>Showing Custom Component as a prop</v-card-title>
                <jupyter-widget :widget="viewer[0]" @row:click.native="console.log('FUUUCKKKK')"></jupyter-widget>
              </v-card>
            </template>
            <script>
            module.exports = {
              props: ["viewer"]
            };
            </script>
        """
    }).tag(sync=True)
    
    some_widget = List([
        test
    ]).tag(sync=True, **w.widget_serialization)
    
Testing()

In [ ]:
b.DataFrame.metadata['to_json'](iris, 2)

In [ ]:


test.clicked_row

In [ ]:
test.observe(fn, 'clicked_row')

In [ ]:
def fn(event):
    print(event)

In [ ]:
iris.iloc[0, :].to_frame().style.bar(align='mid',  color=['#d65f5f', '#5fba7d'], width=100).render()

In [ ]:
a = w.IntText()

In [ ]:
a.class_traits()

In [ ]:
d.observe(fn, '')

In [ ]:
test.df

In [ ]:
import ipywidgets as widgets
import ipyvue as vue
from traitlets import Dict, Unicode, List
import ipyvuetify as v


select_template = '''
    <template>
    <v-row align="center">
        <v-col cols="4">
            <v-select :items="items" label="Test"></v-select>
        </v-col>
    </v-row>
    </template>
    <script>
    module.exports = {
        props: ["items"],
    }
    </script>
'''

class Select(v.VuetifyTemplate):
    items = List([]).tag(sync=True)
    template = Unicode(select_template).tag(sync=True)
    

class DatePicker(v.VuetifyTemplate):
    date = Unicode('2021-03-04').tag(sync=True)
    template = Unicode('''
        <template>
        <v-row align="center">
            <v-col cols="4">
                <v-date-picker v-model="date"></v-date-picker>
            </v-col>
        </v-row>
        </template>
        <scripts>
        module.exports = {
            props: ["date"]
        }
        </scripts>
    ''').tag(sync=True)

In [ ]:
class Main(v.VuetifyTemplate):
    items = List(['a', 'b', 'c']).tag(sync=True)
    date = Unicode('2000-12-12').tag(sync=True)
    template = Unicode('''
        <template>

            <v-card>
                <v-card-title>Showing SubComponents</v-card-title>
                <c-select :items="items"/>
                <c-picker :date="date"/>
            </v-card>
            
        </template>
    ''').tag(sync=True)
    
    components=Dict({
        'c-select': Select,
        'c-picker': DatePicker,
    }).tag(sync=True, **v.VuetifyTemplate.class_component_serialization)

Main(items=['a', 'b', 'c'])

In [ ]:
class Main(v.VuetifyTemplate):
    items = List(['a', 'b', 'c']).tag(sync=True)
    date = Unicode('2000-12-12').tag(sync=True)
    template = Unicode('''
        <template>
        
            <v-card>
                <div style="background-image:linear-gradient(to right, red 20%, orange 20% 40%, yellow 40% 60%, green 60% 80%, blue 80%)"> sdf</div>
                <v-card-title>Showing SubComponents</v-card-title>
                <my-select :items="items"/>

            </v-card>
            
        </template>
    ''').tag(sync=True)
    
    components=Dict({
        'my-select': Select,
    }).tag(sync=True, **v.VuetifyTemplate.class_component_serialization)

Main(items=['a', 'b', 'c'])

In [ ]:
%%html

<div style="background-image:linear-gradient(to right, red 20%, orange 20% 40%, yellow 40% 60%, green 60% 80%, blue 80%)"> sdf</div>